In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs
import scipy.stats as stats

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable

import shapely.geometry as sg
from shapely.geometry import Point

import sys
sys.path.append(os.path.abspath(".."))
from function import DOWN_raw
from function import ART_preprocessing as ART_pre

import warnings
warnings.filterwarnings('ignore')

## ENSEMBLE SAT and DOWN
## Export Quantiles, Weibull Param and Relative Errors for each region as csv

In [2]:
ensemble_product = 'all'
# ensemble_product = 'NoGSMaP'

In [3]:
font = 'SAT'
# font = 'DOWN'

QC_method = 'QCv1_Flag1'

In [4]:
lon_min, lon_max, lat_min, lat_max, area, toll = 6.5, 19, 36.5, 48, 'ITALY', 0.002

Tr = [5,  10,  20,  50, 100, 200]
Fi = 1 - 1/np.array(Tr)

In [5]:
veneto_dir = os.path.join('/','media','arturo','T9','Data','shapes','Europa','Italy')

if os.path.exists(veneto_dir):
    REGIONS = gpd.read_file(os.path.join(veneto_dir,'Italy_regions.geojson'))
else:
    raise SystemExit(f"File not found: {veneto_dir}")

In [6]:
obs_base = os.path.join('/','media','arturo','T9','Data','Italy', 'stations', 'Italy_raingauges', 'QC_MIX')

# sat_base = os.path.join('/','media','arturo','T9','Data','Italy','Satellite','5_DOWN')
sat_base = os.path.join('/','media','arturo','T9','Data','Italy','Satellite','6_DOWN_corrected')

In [7]:
METADATA = pd.read_csv(os.path.join(obs_base, 'METADATA', 'METADATA_INTENSE_UPDATE.csv'))
METADATA["Lat"] = np.round(METADATA["Lat"], 6)
METADATA["Lon"] = np.round(METADATA["Lon"], 6)
METADATA.head(2)

,Unnamed: 0,File,ISO,Region,Code,Name,Elevation,Or_EPSG,CRS_E,CRS_N,...,Source,TimeZon,Elevation.1,Orig_EPSG,Year_Start,Year_End,INTENSE_Resp,TimeZone,elevation_DEM,Elevation_update
0,0,IT-322_AA_6535_0060_QCv1.csv,IT-322,Alto-Adige,AA_6535,VANDOIESDISOPRA,746.0,32632.0,710057.0,5187849.0,...,Alto-Adige,UTC,NaN,NaN,NaN,NaN,NaN,NaN,745.9,746.0
1,1,IT-322_AA_5650_0060_QCv1.csv,IT-322,Alto-Adige,AA_5650,SELVADEIMOLINI,1141.0,32632.0,718972.0,5196967.0,...,Alto-Adige,UTC,NaN,NaN,NaN,NaN,NaN,NaN,1140.0,1141.0


## Load ISO regions and names

In [8]:
ISO_names = pd.read_csv(os.path.join('/','media','arturo','T9','Data','Italy', 'stations', 'Italy_raingauges', 'ISO_IT_REGION_EN.csv'))
ISO_names.head(2)

,Region,ISO
0,Alto Adige,IT-322
1,Aosta Valley,IT-230


## Load ENSEMBLE data

In [9]:
print(f'Load {ensemble_product}')
if ensemble_product == 'all':
    dir_in = os.path.join(sat_base,'ITALY_ENSEMBLE_1dy_2002_2023_npix_2_thr_1_acf_mar_genetic_pearson.nc')
elif ensemble_product == 'all_meadian':
    dir_in = os.path.join(sat_base,'ITALY_ENSEMBLE_1dy_2002_2023_npix_2_thr_1_acf_mar_genetic_pearson_median.nc')
elif ensemble_product == 'NoGSMaP':
    dir_in = os.path.join(sat_base,'ITALY_ENSEMBLE_1dy_2002_2023_npix_2_thr_1_acf_mar_genetic_pearson_NOGSMaP.nc')
else:
    print(f'ensemble_product not found')

data = xr.open_dataset(dir_in)

lat_sat = data.lat.values
lon_sat = data.lon.values
lon2d_sat, lat2d_sat = np.meshgrid(lon_sat, lat_sat)

Sat_year = data.year.values

if font == 'SAT':
    Sat_NYd = data.NYs.values
    Sat_CYd = data.CYs.values
    Sat_WYd = data.WYs.values
elif font == 'DOWN':
    Sat_NYd = data.NYd.values
    Sat_CYd = data.CYd.values
    Sat_WYd = data.WYd.values
else:
    print('Font dont found')
    sys.exit()

Load all


## Choose the Region

In [10]:
Tr_index = 3
print(f'Tr: {Tr[Tr_index]} years')

Tr: 50 years


In [11]:
ISO_names["ISO_num"] = ISO_names["ISO"].str.split("-").str[1].astype(int)
ISO_names = ISO_names.sort_values("ISO_num").drop(columns="ISO_num").reset_index(drop=True)
ISO_names.head(5)

,Region,ISO
0,Piedmont,IT-210
1,Aosta Valley,IT-230
2,Lombardy,IT-250
3,Trentino,IT-321
4,Alto Adige,IT-322


In [12]:
# # Strange Q50 Values
# coords_remove = [
#     (13.180000, 43.310000),
#     (11.202635, 42.434474),
#     (11.548882, 42.582992),
#     (9.53778, 45.01694)
# ]

# METADATA["coord"] = list(zip(METADATA["Lon"], METADATA["Lat"]))
# METADATA = METADATA[~METADATA["coord"].isin(coords_remove)]
# METADATA = METADATA.drop(columns="coord")
# METADATA = METADATA.reset_index(inplace=False)

# METADATA.to_csv(os.path.join(obs_base, 'METADATA', 'METADATA_INTENSE_UPDATE.csv'))

In [13]:
diccionario_OBS_WN = {}
diccionario_OBS_WC = {}
diccionario_OBS_WW = {}

diccionario_SAT_WN = {}
diccionario_SAT_WC = {}
diccionario_SAT_WW = {}

diccionario_RE = {}
diccionario_LAT = {}
diccionario_LON = {}
diccionario_ELEV = {}

diccionario_Qobs = {}
diccionario_Qest = {}

for rr in range(len(ISO_names)):#len(ISO_names)
    region_ISO = ISO_names.iloc[rr]['ISO']
    region_label = ISO_names[ISO_names['ISO']==region_ISO]

    print(f'{rr+1}: {region_label['Region'].values[0]} ({region_ISO})')

    METADATA_clear = METADATA[METADATA['ISO']==region_ISO].reset_index(inplace=False)

    SAT_WN, SAT_WC, SAT_WW = [], [], []
    OBS_WN, OBS_WC, OBS_WW = [], [], []
    LAT, LON, RE_50, ELEV = [], [], [], []
    Qobs, Qest = [], []
    for nn in range(len(METADATA_clear)):#len(METADATA_clear)
        filename = f'{METADATA_clear['File'].values[nn]}'
        lat_obs = METADATA_clear['Lat'][nn]
        lon_obs = METADATA_clear['Lon'][nn]
        elev_obs = METADATA_clear['Elevation_update'][nn]

        OBS_pd = pd.read_csv(os.path.join(obs_base, 'DATA_1dy', 'statistics', QC_method, region_ISO, filename))
        OBS_pd = OBS_pd[(OBS_pd['Year']>=2002)&(OBS_pd['Year']<=2023)].reset_index(drop=True)
        if len(OBS_pd) == 0:
            continue
        
        else:
            OBS_N = OBS_pd['N'].values
            OBS_C = OBS_pd['C'].values
            OBS_W = OBS_pd['W'].values
            OBS_Y = OBS_pd['Year'].values

            mask = ~np.isnan(OBS_N)

            OBS_N = OBS_N[mask]
            OBS_C = OBS_C[mask]
            OBS_W = OBS_W[mask]
            OBS_Y = OBS_Y[mask]

            if len(OBS_Y) >= 8: # greather than 8 years

                x0 = np.nanmean(OBS_C)
                OBS_Q, flag = DOWN_raw.mev_quant_update(Fi, x0, OBS_N, OBS_C, OBS_W, thresh=1)
                OBS_Q2 = np.where(flag, OBS_Q, np.nan)

                distances = ART_pre.haversine(lat2d_sat, lon2d_sat, lat_obs, lon_obs)
                min_index = np.unravel_index(np.argmin(distances, axis=None), distances.shape)
                Sat_NYd_ = Sat_NYd[:,min_index[0],min_index[1]]
                Sat_CYd_ = Sat_CYd[:,min_index[0],min_index[1]]
                Sat_WYd_ = Sat_WYd[:,min_index[0],min_index[1]]

                Sat_pd = pd.DataFrame({'Year':Sat_year, 'N':Sat_NYd_, 'C':Sat_CYd_, 'W':Sat_WYd_})
                # Sat_pd_filtered = Sat_pd[Sat_pd['Year'].isin(OBS_pd['Year'])].reset_index()
                Sat_pd_filtered = Sat_pd.set_index('Year').loc[OBS_pd['Year']].reset_index()

                SAT_N = Sat_pd_filtered['N'].values
                SAT_C = Sat_pd_filtered['C'].values
                SAT_W = Sat_pd_filtered['W'].values
                SAT_Y = Sat_pd_filtered['Year'].values

                SAT_N = SAT_N[mask]
                SAT_C = SAT_C[mask]
                SAT_W = SAT_W[mask]
                SAT_Y = SAT_Y[mask]

                if len(OBS_Y) != len(SAT_Y):
                    print('ERROR')
                    sys.exit()
                else:
                    
                    OBS_WN.extend(OBS_N)
                    OBS_WC.extend(OBS_C)
                    OBS_WW.extend(OBS_W)
                    
                    SAT_WN.extend(SAT_N)
                    SAT_WC.extend(SAT_C)
                    SAT_WW.extend(SAT_W)
                    
                    x0 = np.nanmean(SAT_C)
                    SAT_Q, flag = DOWN_raw.mev_quant_update(Fi, x0, SAT_N, SAT_C, SAT_W, thresh=1)
                    SAT_Q2 = np.where(flag, SAT_Q, np.nan)

                relative_e_50 = (SAT_Q2[3] - OBS_Q2[3])/OBS_Q2[3]

                LAT.append(float(lat_obs))
                LON.append(float(lon_obs))
                RE_50.append(float(relative_e_50))
                ELEV.append(float(elev_obs))
                Qobs.append(float(OBS_Q2[3]))
                Qest.append(float(SAT_Q2[3]))

            else:        
                continue

            RE_50_M = np.array(RE_50)
            # RE_50_M = RE_50_M[(RE_50_M>-1)&(RE_50_M<1)]
            diccionario_RE[region_ISO] = RE_50_M

            LAT_M = np.array(LAT)
            # LAT_M = LAT_M[(np.array(RE_50)>-1)&(np.array(RE_50)<1)]
            diccionario_LAT[region_ISO] = LAT_M

            LON_M = np.array(LON)
            # LON_M = LON_M[(np.array(RE_50)>-1)&(np.array(RE_50)<1)]
            diccionario_LON[region_ISO] = LON_M

            ELEV_M = np.array(ELEV)
            # ELEV_M = ELEV_M[(np.array(RE_50)>-1)&(np.array(RE_50)<1)]
            diccionario_ELEV[region_ISO] = ELEV_M

            Qobs_M = np.array(Qobs)
            # Qobs_M = Qobs_M[(np.array(RE_50)>-1)&(np.array(RE_50)<1)]
            diccionario_Qobs[region_ISO] = Qobs_M

            Qest_M = np.array(Qest)
            # Qest_M = Qest_M[(np.array(RE_50)>-1)&(np.array(RE_50)<1)]
            diccionario_Qest[region_ISO] = Qest_M

            OBS_WN_N = np.array(OBS_WN)
            diccionario_OBS_WN[region_ISO] = OBS_WN_N
            
            OBS_WC_N = np.array(OBS_WC)
            diccionario_OBS_WC[region_ISO] = OBS_WC_N
            
            OBS_WW_N = np.array(OBS_WW)
            diccionario_OBS_WW[region_ISO] = OBS_WW_N

            SAT_WN_N = np.array(SAT_WN)
            diccionario_SAT_WN[region_ISO] = SAT_WN_N
            
            SAT_WC_N = np.array(SAT_WC)
            diccionario_SAT_WC[region_ISO] = SAT_WC_N
            
            SAT_WW_N = np.array(SAT_WW)
            diccionario_SAT_WW[region_ISO] = SAT_WW_N


1: Piedmont (IT-210)
2: Aosta Valley (IT-230)
3: Lombardy (IT-250)
4: Trentino (IT-321)
5: Alto Adige (IT-322)
6: Veneto (IT-340)
7: Friuli-Venezia Giulia (IT-360)
8: Liguria (IT-420)
9: Emilia-Romagna (IT-450)
10: Tuscany (IT-520)
11: Umbria (IT-550)
12: Marche (IT-570)
13: Lazio (IT-620)
14: Abruzzo (IT-650)
15: Molise (IT-670)
16: Campania (IT-720)
17: Apulia (IT-750)
18: Basilicata (IT-770)
19: Calabria (IT-780)
20: Sicily (IT-820)
21: Sardinia (IT-880)


## Extreme Quantiles Tr=50 yrs

In [14]:
diccionario_Qobs = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_Qobs.items()]))
diccionario_Qest = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_Qest.items()]))

## Weibull Parameters

In [15]:
diccionario_OBS_WN = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_OBS_WN.items()]))
diccionario_OBS_WC = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_OBS_WC.items()]))
diccionario_OBS_WW = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_OBS_WW.items()]))

## Relative Error and Position

In [16]:
diccionario_SAT_WN = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_SAT_WN.items()]))
diccionario_SAT_WC = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_SAT_WC.items()]))
diccionario_SAT_WW = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_SAT_WW.items()]))

diccionario_RE = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_RE.items()]))
diccionario_LAT = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_LAT.items()]))
diccionario_LON = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_LON.items()]))
diccionario_ELEV = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in diccionario_ELEV.items()]))

In [17]:
diccionario_Qobs[diccionario_Qobs>500]=np.nan
diccionario_Qest[diccionario_Qobs>500]=np.nan

diccionario_OBS_WN[diccionario_Qobs>500]=np.nan
diccionario_OBS_WC[diccionario_Qobs>500]=np.nan
diccionario_OBS_WW[diccionario_Qobs>500]=np.nan

diccionario_SAT_WN[diccionario_Qobs>500]=np.nan
diccionario_SAT_WC[diccionario_Qobs>500]=np.nan
diccionario_SAT_WW[diccionario_Qobs>500]=np.nan

diccionario_RE[diccionario_Qobs>500]=np.nan
diccionario_LAT[diccionario_Qobs>500]=np.nan
diccionario_LON[diccionario_Qobs>500]=np.nan
diccionario_ELEV[diccionario_Qobs>500]=np.nan

## Export DataFrames as csv

In [18]:
dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'OBS_Q50.csv')
print(f'Export as: {dir_out}')
diccionario_Qobs.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'{font}_ensemble_Q50.csv')
print(f'Export as: {dir_out}')
diccionario_Qest.to_csv(dir_out, header=True, index=False)

Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/OBS_Q50.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/SAT_ensemble_Q50.csv


In [19]:
dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'OBS_Weibull_N.csv')
print(f'Export as: {dir_out}')
diccionario_OBS_WN.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'OBS_Weibull_C.csv')
print(f'Export as: {dir_out}')
diccionario_OBS_WC.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'OBS_Weibull_W.csv')
print(f'Export as: {dir_out}')
diccionario_OBS_WW.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'{font}_Elev.csv')
print(f'Export as: {dir_out}')
diccionario_ELEV.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'OBS_relative_error_lat.csv')
print(f'Export as: {dir_out}')
diccionario_LAT.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'OBS_relative_error_lon.csv')
print(f'Export as: {dir_out}')
diccionario_LON.to_csv(dir_out, header=True, index=False)

Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/OBS_Weibull_N.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/OBS_Weibull_C.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/OBS_Weibull_W.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/SAT_Elev.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/OBS_relative_error_lat.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/OBS_relative_error_lon.csv


In [20]:
dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'{font}_ensemble_Weibull_N.csv')
print(f'Export as: {dir_out}')
diccionario_SAT_WN.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'{font}_ensemble_Weibull_C.csv')
print(f'Export as: {dir_out}')
diccionario_SAT_WC.to_csv(dir_out, header=True, index=False)

dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'{font}_ensemble_Weibull_W.csv')
print(f'Export as: {dir_out}')
diccionario_SAT_WW.to_csv(dir_out, header=True, index=False)

Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/SAT_ensemble_Weibull_N.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/SAT_ensemble_Weibull_C.csv
Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/SAT_ensemble_Weibull_W.csv


In [21]:
dir_out = os.path.join(obs_base, 'DATA_1dy', 'RE_ENSEMBLE_SAT', f'{font}_ensemble_relative_error.csv')
print(f'Export as: {dir_out}')
diccionario_RE.to_csv(dir_out, header=True, index=False)

Export as: /media/arturo/T9/Data/Italy/stations/Italy_raingauges/QC_MIX/DATA_1dy/RE_ENSEMBLE_SAT/SAT_ensemble_relative_error.csv
